<a href="https://colab.research.google.com/drive/1vztM9vNv8IWWwPwonOO35VqNkJjktp4B?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

### [CrewAI](https://docs.crewai.com/introduction) based Multi-Agentic system - AI Research Assistant

This implementation uses CrewAI to create a powerful research system with three specialized agents working together to deliver comprehensive research reports:

1. Research Analyst: Conducts thorough research on topics using Serper for web search
2. Data Analyst: Analyzes findings to extract insights, patterns, and strategic recommendations
3. Content Writer: Creates clear, engaging content in a professional business report format

#### Setup
1. Install required packages: `pip install crewai crewai_tools`
2. Set your OpenAI API key (required)
3. Set your Serper API key for web search capabilities

#### Usage
Configure your research by specifying topic, type (General/Market/Technical/Trend), depth level, and specific requirements. The system handles delegation between agents and delivers a structured final report with analysis and recommendations.

Example: `run_research(specific_requirements="...", research_topic="Impact of AI on Healthcare", research_type="Market Analysis")`

### Install required libraries

In [ ]:
!pip install -q -U crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 28.1 MB/s

### Import related libraries

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

In [ ]:
import getpass
import os

#### Provide a OpenAI API key (Paid One)

[OpenAI API Key Creation Link](https://platform.openai.com/api-keys)




In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


#### Provide a Serper API key

[Serper API Key Creation Link](https://serper.dev/api-key)

In [ ]:
os.environ["SERPER_API_KEY"] = getpass.getpass()

··········


In [ ]:
def create_research_crew():
    """Create the research crew with agents"""

    search_tool = SerperDevTool(
        search_url="https://google.serper.dev/search",
        n_results=2,
    )

    researcher = Agent(
        role='Research Analyst',
        goal='Conduct thorough research on topics',
        backstory='Expert researcher with years of experience',
        verbose=False,
        tools=[search_tool],
        allow_delegation=True
    )

    analyst = Agent(
        role='Data Analyst',
        goal='Analyze findings and extract insights',
        backstory='Experienced data analyst and interpreter',
        verbose=False,
        allow_delegation=True
    )

    writer = Agent(
        role='Content Writer',
        goal='Create clear, engaging content',
        backstory='Professional technical writer',
        verbose=False,
        allow_delegation=True
    )

    return researcher, analyst, writer

In [ ]:
def create_tasks(researcher, analyst, writer, research_topic, research_type,
                depth_level, requirements):
    """Create research tasks"""
    tasks = [
        Task(
            description=f"""
            Research the topic: {research_topic}
            Type: {research_type}
            Depth: {depth_level}
            Requirements: {requirements}
            """,
            expected_output="""A comprehensive research document containing:
            - Detailed findings and discoveries
            - Statistical data and trends
            - Key stakeholder information
            - Market/industry context
            - Relevant sources and citations""",
            agent=researcher
        ),
        Task(
            description=f"""
            Analyze the research findings with focus on:
            1. Key patterns and trends
            2. Impact assessment
            3. Comparative analysis
            4. Future projections
            Depth Level: {depth_level}
            """,
            expected_output="""An analytical report containing:
            - Key trends and patterns identified
            - Impact analysis and implications
            - Data-driven insights
            - Strategic recommendations
            - Risk assessment""",
            agent=analyst
        ),
        Task(
            description=f"""
            Create a comprehensive report synthesizing the research and analysis.
            Research Type: {research_type}
            Depth: {depth_level}
            Format: Professional business report
            """,
            expected_output="""A well-structured final report including:
            - Executive summary
            - Key findings and insights
            - Detailed analysis
            - Supporting data and visuals
            - Conclusions and recommendations
            - References and sources""",
            agent=writer
        )
    ]
    return tasks

In [ ]:
# Default Research Values
DEFAULT_RESEARCH_TOPIC = "Cryptocurrency and DeFi Trends in Emerging Markets"
DEFAULT_REQUIREMENTS = """1. Adoption rates in different regions
2. Regulatory landscape
3. Popular platforms and services
4. Financial inclusion impact
5. Market growth projections"""

def run_research(
    specific_requirements,
    research_topic: str = DEFAULT_RESEARCH_TOPIC,
    research_type: str = "General Research",
    depth_level: str = "Moderate",

):
    """
    Run the research process with the specified parameters

    Args:
        research_topic (str): The topic to research
        research_type (str): Type of research (General Research, Market Analysis, Technical Review, Trend Analysis)
        depth_level (str): Depth of research (Basic, Moderate, Comprehensive, Expert)
        specific_requirements (str): Specific requirements for the research

    Returns:
        str: The research results
    """
    try:
        print("Initializing research crew...")
        researcher, analyst, writer = create_research_crew()

        tasks = create_tasks(
            researcher, analyst, writer,
            research_topic, research_type,
            depth_level, specific_requirements
        )

        crew = Crew(
            agents=[researcher, analyst, writer],
            tasks=tasks,
            verbose=False
        )

        print("Conducting research...")
        crew_result = crew.kickoff()
        result = str(crew_result)

        print("Research completed!")
        return result

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check if all required API keys are correctly configured.")
        return None


In [ ]:
# Set your API keys in environment variables or .env file first
research_result = run_research(
    specific_requirements='''
    1. Current AI applications in healthcare
    2. Market size and growth projections
    3. Key players and technologies
    4. Regulatory considerations
    5. Implementation challenges
    ''',
    research_topic="Impact of AI on Healthcare in 2024",
    research_type="Market Analysis",
    depth_level="Comprehensive",
)

print(research_result)

Initializing research crew...
Conducting research...
Research completed!
# Comprehensive Market Analysis: The Impact of AI on Healthcare in 2024

## Executive Summary
As we enter 2024, Artificial Intelligence (AI) continues to revolutionize the healthcare landscape, enhancing patient care, streamlining operations, and driving innovative research. This analysis delves deep into the current applications of AI in healthcare, examining the market size, growth projections, key players, regulatory considerations, and implementation challenges faced by healthcare organizations.

## 1. Current AI Applications in Healthcare
AI technologies are being deployed across various sectors within healthcare. Some key applications include:

### 1.1. Diagnostic Tools
AI algorithms are widely used in diagnostic imaging, pathology, and genomics. For instance, platforms like Google's DeepMind have been deployed for early detection of diseases such as diabetic retinopathy and age-related macular degeneration.